## Taller de cuantificación: Estimación de prevalencias por clase
Pontentes: Pablo Gonzalez (gonzalezgpablo@uniovi.es) y Olaya Pérez (perezolaya@uniovi.es)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pglez82/quantificationlib_lab/blob/master/lab/lab.ipynb)

### Parte 1: Descarga del dataset necesario

In [ ]:
import os
import zipfile

zip_url = "https://zenodo.org/records/11661820/files/T1.train_dev.zip?download=1"
zip_path = "T1.train_dev.zip"
extracted_folder = "T1.train_dev"

# Download with system wget (Jupyter syntax)
if not os.path.exists(zip_path):
    print("Downloading dataset...")
    !wget -O $zip_path "$zip_url"
else:
    print("ZIP file already exists.")

# Extract if not already extracted
if not os.path.exists(extracted_folder):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_folder)
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

### Parte 2: Instalación de las librerías necesarias

In [ ]:
!pip install pandas
!pip install matplotlib
!pip install quantificationlib
!pip install scikit-learn

### Parte 3. Leyendo el dataset.
Este dataset contiene reviews de productos de amazon. Cada review puede ser positiva o negativa (campo `label`). El texto de cada review ha sido convertido a una representación numérica para su facilidad de uso. Diponemos de 5000 ejemplos de entrenamiento (5000 opiniones).

In [ ]:
import pandas as pd

dataset = pd.read_csv('T1.train_dev/T1/public/training_data.txt')

print(dataset)

### Analizando la prevalencia de las clases original en el dataset de etrenamiento.
Como podemos ver, un alto porcentaje de las opiniones son positivas.

In [ ]:
import matplotlib.pyplot as plt

# Count examples per label
label_counts = dataset['label'].value_counts()

# Plot
plt.figure(figsize=(5, 4))
plt.bar(label_counts.index.astype(str), label_counts.values)
plt.xlabel('Etiqueta (0 negativa, 1 positiva)')
plt.ylabel('Número de ejemplos')
plt.title('Distribución')

plt.grid(axis='y')
plt.show()


### Aprendiendo a cuantificar. El método Clasificar y Contar
Vamos a empezar con la solución trivial del problema de la cuantificación: entrenar un clasificador y contar las predicciones de cada ejemplo de la bag de test que queremos cuantificar.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

X = dataset.drop(columns=["label"]).values
y = dataset["label"].values

#Dividimos nuestro dataset en train y test, separando el 30% de los ejemplos para test y asegurando que la proporción original de etiquetas se mantiene en ambos conjuntos.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Train: {len(y_train)} ejemplos (Proporción positivos: {np.mean(y_train):.2f}) | Test: {len(y_test)} ejemplos (Proporción positivos: {np.mean(y_test):.2f})")

Entrenamos un clasificador y clasificamos lo ejemplos de test. Como podemos ver, el error absoluto de este cuantificador (AE) es bajo, predice bastante bien la prevalencia de la clse positiva en la bag de test. Este problema es en realidad trivial ya que en nuestro setup la prevalencia de las clases **no ha cambiado** entre train y test: $P_{train}(y)=P_{test}(y)$

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# Predeccimos los ejemplos de test y contamos la proporción de positivos en las predicciones.
cc_prevalence = np.mean(y_pred)
true_prevalence = np.mean(y_test)

print(f"Clasificar y contar (porcentaje postivos): {cc_prevalence:.4f}")
print(f"Porcentaje positivos real: {true_prevalence:.4f}")
print(f"Error absoluto cuantificador: {np.abs(cc_prevalence-true_prevalence):.4f}")

### ¿Qué sucede si la distribución $P(y)$ cambia entre train y test $P_{train}(y) \neq P_{test}(y)$?
Para probarlo, generamos una nueva bolsa de test con 500 ejemplos, pero en este caso, la mayoría van a ser negativos (400 de 500).
Como podemos comprobar ahora, si repetimos el procedimiento de antes y utilizamos CC para cuantificar, el error absoluto entre la predicción del cuantificador y la prevalencia real es mucho mayor.

In [ ]:
# Calculamos los índices de los ejemplos negativos y positivos en el conjunto de test
neg_indices = np.where(y_test == 0)[0]
pos_indices = np.where(y_test == 1)[0]

# Elegimos el número de ejemplos negativos y positivos que queremos en nuestra bag
n_neg, n_pos = 400, 100

# Muestreamos aleatoriamente con reemplazmiento los índices de los ejemplos negativos y positivos
sampled_neg = np.random.choice(neg_indices, size=n_neg, replace=True)
sampled_pos = np.random.choice(pos_indices, size=n_pos, replace=True)

# Combinamos y barajamos los índices de la bag
bag_indices = np.concatenate([sampled_neg, sampled_pos])
np.random.shuffle(bag_indices)

# Creamos la bag con los ejemplos seleccionados
X_bag = X_test[bag_indices]
y_bag = y_test[bag_indices]

# Imprimir información sobre la bag
print(f"Número de ejemplos en test: {len(y_bag)} | Proporción positivos: {np.mean(y_bag):.1f}")

# Aplicamos el método clasificar y contar (CC) a la bag
y_pred = clf.predict(X_bag)
cc_prevalence = np.mean(y_pred)
true_prevalence = np.mean(y_bag)

print(f"Clasificar y contar (porcentaje positivos): {cc_prevalence:.4f}")
print(f"Porcentaje positivos real: {true_prevalence:.2f}")
print(f"Error absoluto cuantificador: {np.abs(cc_prevalence-true_prevalence):.4f}")


### Automatizando la cuantificación (quantificationlib)

Hasta ahora hemos cuantificado, con un cuantificador trivial (CC) a mano. Para cuantificar disponemos de software específico, como por ejemplo la librería de cuantificación **quantificationlib** (https://aicgijon.github.io/quantificationlib/).

Veamos como utilizar CC usando quantificationlib.

In [ ]:
from quantificationlib.baselines.cc import CC
from quantificationlib.metrics.multiclass import mean_absolute_error

# Creamos un objeto CC que corresponde al método clasificar y contar (CC) e indicamos que queremos el mismo clasificador que hemos utilizado antes (clf).
quantifier_cc = CC(estimator_test=clf)
quantifier_cc.fit(X_train,y_train)
# Predeccimos la misma bag con el cuantificador CC (deberíamos de obtener los mismos resultados que antes)
p_pred=quantifier_cc.predict(X_bag)

true_prevalence = np.array([0.8,0.2])

print(f"Clasificar y contar (porcentaje postivos): {p_pred[1]}")
print(f"Prevalencia real: {true_prevalence[1]}")
# También disponemos de funciones para calcular errores de cuantificación como el error absoluto
print(f"Error absoluto cuantificador: {mean_absolute_error(p_pred,true_prevalence)}")

### Utilizando cuantificadores más avanzados
En este caso vamos a utilizar un cuantificador especialmente diseñador para lidiar con situaciones de prior probability shift. Este método es un cuantificador básico conocido como Adjusted Count (AC).

Como podemos ver este cuantificador mejora muchísimo el resultado de CC.

In [ ]:
from quantificationlib.baselines.ac import AC

quantifier_ac = AC(estimator_train=clf, estimator_test=clf)
quantifier_ac.fit(X_train,y_train)
p_pred=quantifier_ac.predict(X_bag)

print(f"Adjusted count (porcentaje postivos): {p_pred[1]}")
print(f"Prevalencia real: {true_prevalence[1]}")
print(f"Error absoluto cuantificador: {mean_absolute_error(p_pred,true_prevalence)}")

### Evaluando correctamente un cuantificador

Hasta ahora hemos hecho nuestras pruebas sobre una sola bag, con una prevalencia fija. Pero, no sería más adecuado crear varias bags con prior probability shift y calcular el error que cometemos? Esto se puede realizar fácilmente con un generador de bags incluido en **quantificationlib**. Veamos como se usa.

In [ ]:
from quantificationlib.bag_generator import PriorShift_BagGenerator

n_bags = 10
bag_size = 500

bag_generator = PriorShift_BagGenerator(n_bags=n_bags, bag_size=bag_size, min_prevalence=None)
prev_true, indexes = bag_generator.generate_bags(X_test, y_test)
errores_cc = np.zeros((n_bags))
errores_ac = np.zeros((n_bags))

for n_bag in range(10):
   prev_pred_cc = quantifier_cc.predict(X_test[indexes[:, n_bag], :])
   prev_pred_ac = quantifier_ac.predict(X_test[indexes[:, n_bag], :])
   errores_cc[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_cc)
   errores_ac[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_ac)
   print('Prevalencia real=%f, CC=%f (AE=%f), AC=%f (AE=%f)' % (prev_true[1,n_bag],prev_pred_cc[1], errores_cc[n_bag] ,prev_pred_ac[1], errores_ac[n_bag]))

print(f"Error medio CC {np.mean(errores_cc):.4f}")
print(f"Error medio AC {np.mean(errores_ac):.4f}")


### Ejercicio 1: Entrena dos cuantificadores más. El primero basado en distribution matching (DFy) y el segundo basado en el agoritmo EM (EM), compara los resultados con AC y CC.

In [ ]:
from quantificationlib.bag_generator import PriorShift_BagGenerator
from quantificationlib.multiclass.em import EM
from quantificationlib.multiclass.df import DFy

n_bags = 10
bag_size = 500

quantifier_dfy = DFy(estimator_train=clf, estimator_test=clf)
quantifier_em = EM(estimator_train=clf, estimator_test=clf)
quantifier_dfy.fit(X_train,y_train)
quantifier_em.fit(X_train,y_train)

bag_generator = PriorShift_BagGenerator(n_bags=n_bags, bag_size=bag_size, min_prevalence=None)
prev_true, indexes = bag_generator.generate_bags(X_test, y_test)
errores_cc = np.zeros((n_bags))
errores_ac = np.zeros((n_bags))
errores_dfy = np.zeros((n_bags))
errores_em = np.zeros((n_bags))

for n_bag in range(10):
   prev_pred_cc = quantifier_cc.predict(X_test[indexes[:, n_bag], :])
   prev_pred_ac = quantifier_ac.predict(X_test[indexes[:, n_bag], :])
   prev_pred_dfy = quantifier_dfy.predict(X_test[indexes[:, n_bag], :])
   prev_pred_em = quantifier_em.predict(X_test[indexes[:, n_bag], :])
   errores_cc[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_cc)
   errores_ac[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_ac)
   errores_dfy[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_dfy)
   errores_em[n_bag] = mean_absolute_error(prev_true[:,n_bag],prev_pred_em)
   print('Prevalencia real=%f, CC=%f (AE=%f), AC=%f (AE=%f), DFy=%f (AE=%f), EM=%f (AE=%f)' % (prev_true[1,n_bag],prev_pred_cc[1], errores_cc[n_bag] ,prev_pred_ac[1], errores_ac[n_bag], prev_pred_dfy[1], errores_dfy[n_bag], prev_pred_em[1], errores_em[n_bag]))

print(f"Error medio CC {np.mean(errores_cc):.4f}")
print(f"Error medio AC {np.mean(errores_ac):.4f}")
print(f"Error medio DFy {np.mean(errores_dfy):.4f}")
print(f"Error medio EM {np.mean(errores_em):.4f}")

### Ejercicio 2: Utiliza los 5000 ejemplos de entrenamiento para reentrenar tus cuatro cuantificadores. Usa estos cuantificadores para cuantificar las 1000 muestras presentes en el directorio T1.train_dev/T1/public/dev_samples.
La prevalencia real de cada bag está en el archivo dev_prevalences.txt

In [ ]:
# leer las muestras del directorio T1.train_dev/T1/public/dev_samples
import os
import pandas as pd
dev_samples_path = 'T1.train_dev/T1/public/dev_samples'
# Leer la prevalencia real de cada bag desde el archivo dev_prevalences.txt
prev_true = pd.read_csv('T1.train_dev/T1/public/dev_prevalences.txt',index_col=0)
bag_files = prev_true.index
prev_true = prev_true.to_numpy()

clf = LogisticRegression(max_iter=1000)
clf.fit(X, y)
quantifier_cc.fit(X,y)
quantifier_ac.fit(X,y)
quantifier_dfy.fit(X,y)
quantifier_em.fit(X,y)
errores_cc = np.zeros((1000))
errores_ac = np.zeros((1000))
errores_dfy = np.zeros((1000))
errores_em = np.zeros((1000))
for n_bag in bag_files:
    file_path = os.path.join(dev_samples_path, str(n_bag) + '.txt')
    df = pd.read_csv(file_path)
    prev_pred_cc = quantifier_cc.predict(df.to_numpy())
    prev_pred_ac = quantifier_ac.predict(df.to_numpy())
    prev_pred_dfy = quantifier_dfy.predict(df.to_numpy())
    prev_pred_em = quantifier_em.predict(df.to_numpy())
    errores_cc[n_bag] = mean_absolute_error(prev_true[n_bag,:],prev_pred_cc)
    errores_ac[n_bag] = mean_absolute_error(prev_true[n_bag,:],prev_pred_ac)
    errores_dfy[n_bag] = mean_absolute_error(prev_true[n_bag,:],prev_pred_dfy)
    errores_em[n_bag] = mean_absolute_error(prev_true[n_bag,:],prev_pred_em)

print(f"Error medio CC {np.mean(errores_cc):.4f}")
print(f"Error medio AC {np.mean(errores_ac):.4f}")
print(f"Error medio DFy {np.mean(errores_dfy):.4f}")
print(f"Error medio EM {np.mean(errores_em):.4f}")